In [1]:
import json 
import numpy as np 
import shutil 
import cv2 
import os 
from pathlib import Path 

### 只处理矩形和旋转框

In [7]:
import numpy as np 
from decimal import Decimal, getcontext 
from shapely.geometry  import Polygon 
 
getcontext().prec = 12  # 12位十进制精度 

def ensure_clockwise(points):
    edge_sum = sum(
        (points[i+1][0]-points[i][0]) * (points[i+1][1]+points[i][1]) 
        for i in range(3)
    )
    return points if edge_sum > 0 else points[::-1]

def convert_to_yolov8(image_path, json_path, output_dir, class_list):
    # 初始化校验 
    img = cv2.imread(image_path) 
    assert img is not None, f"图像加载失败: {image_path}"
    real_h, real_w = img.shape[:2] 
    
    # 读取并净化标注 
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f) 
    
    img_h = float(Decimal(str(data.get("imageHeight",  real_h))))
    img_w = float(Decimal(str(data.get("imageWidth",  real_w))))
    
    # 尺寸修正 
    if abs(img_w - real_w) > 1 or abs(img_h - real_h) > 1:
        print(f"尺寸修正: {img_w}x{img_h} -> {real_w}x{real_h}")
        img_h, img_w = real_h, real_w 
    
    txt_lines = []
    for shape in data["shapes"]:
        # 标签处理 
        label = shape["label"].strip().lower().split(':')[0]
        if label not in class_list:
            continue 
        
        # 坐标净化 
        points = np.array([ 
            [float(Decimal(str(x))), float(Decimal(str(y)))] 
            for x,y in shape["points"]
        ], dtype=np.float64) 
        
        # 矩形框处理 
        if shape["shape_type"] == "rectangle":
            # 获取原始矩形坐标 
            x_coords = points[:,0]
            y_coords = points[:,1]
            
            # 计算OBB顶点（支持带旋转的矩形）
            obb_points = np.array([ 
                [min(x_coords), min(y_coords)],  # 左上 
                [max(x_coords), min(y_coords)],  # 右上 
                [max(x_coords), max(y_coords)],  # 右下 
                [min(x_coords), max(y_coords)]   # 左下 
            ])
            
            # 顶点顺序标准化 
            obb_points = ensure_clockwise(obb_points)
            
            # 归一化处理 
            norm_points = obb_points / np.array([img_w,  img_h])
            txt_lines.append( 
                f"{class_list.index(label)}  " +
                " ".join(f"{p:.8f}" for p in norm_points.flatten()) 
            )
        
        # 旋转框处理 
        elif shape["shape_type"] == "rotation":
            poly = Polygon(points).convex_hull 
            if not poly.is_valid: 
                poly = poly.buffer(0) 
            
            # 获取有序顶点 
            ordered_points = np.array(poly.exterior.coords)[:-1] 
            if len(ordered_points) != 4:
                ordered_points = cv2.boxPoints(cv2.minAreaRect(ordered_points)) 
            
            # 强制顺时针 
            edge_sum = sum(
                (ordered_points[i+1][0] - ordered_points[i][0]) * 
                (ordered_points[i+1][1] + ordered_points[i][1]) 
                for i in range(3)
            )
            if edge_sum <= 0:
                ordered_points = ordered_points[::-1]
            
            # 归一化 
            norm_points = ordered_points / np.array([img_w,  img_h])
            txt_lines.append( 
                f"{class_list.index(label)}  " + 
                " ".join(f"{p:.8f}" for p in norm_points.flatten()) 
            )
    
    # 写入文件 
    os.makedirs(output_dir,  exist_ok=True)
    txt_path = os.path.join(output_dir,  Path(image_path).stem + ".txt")
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(txt_lines)) 
    

### 直接对多边形+矩形+旋转框进行处理

In [5]:
import numpy as np 
from decimal import Decimal, getcontext 
from shapely.geometry  import Polygon 
 
getcontext().prec = 12  # 12位十进制精度 

def ensure_clockwise(points):
    edge_sum = sum(
        (points[i+1][0]-points[i][0]) * (points[i+1][1]+points[i][1]) 
        for i in range(3)
    )
    return points if edge_sum > 0 else points[::-1]

def convert_to_yolov8(image_path, json_path, output_dir, class_list):
    # 初始化校验 
    img = cv2.imread(image_path) 
    assert img is not None, f"图像加载失败: {image_path}"
    real_h, real_w = img.shape[:2] 
    
    # 读取并净化标注 
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f) 
    
    img_h = float(Decimal(str(data.get("imageHeight", real_h))))
    img_w = float(Decimal(str(data.get("imageWidth", real_w))))
    
    # 尺寸修正 
    if abs(img_w - real_w) > 1 or abs(img_h - real_h) > 1:
        print(f"尺寸修正: {img_w}x{img_h} -> {real_w}x{real_h}")
        img_h, img_w = real_h, real_w 
    
    txt_lines = []
    for shape in data["shapes"]:
        # 标签处理 
        label = shape["label"].strip().lower().split(':')[0]
        if label not in class_list:
            continue 
        
        # 坐标净化 
        points = np.array([ 
            [float(Decimal(str(x))), float(Decimal(str(y)))] 
            for x,y in shape["points"]
        ], dtype=np.float64)
        
        # 矩形框处理 
        if shape["shape_type"] == "rectangle":
            # 获取原始矩形坐标 
            x_coords = points[:,0]
            y_coords = points[:,1]
            
            # 计算OBB顶点（支持带旋转的矩形）
            obb_points = np.array([ 
                [min(x_coords), min(y_coords)],  # 左上 
                [max(x_coords), min(y_coords)],  # 右上 
                [max(x_coords), max(y_coords)],  # 右下 
                [min(x_coords), max(y_coords)]   # 左下 
            ])
            
            # 顶点顺序标准化 
            obb_points = ensure_clockwise(obb_points)
            
            # 归一化处理 
            norm_points = obb_points / np.array([img_w, img_h])
            txt_lines.append( 
                f"{class_list.index(label)} " +
                " ".join(f"{p:.8f}" for p in norm_points.flatten()) 
            )
        
        # 旋转框处理 
        elif shape["shape_type"] == "rotation":
            poly = Polygon(points).convex_hull 
            if not poly.is_valid: 
                poly = poly.buffer(0) 
            
            # 获取有序顶点 
            ordered_points = np.array(poly.exterior.coords)[:-1] 
            if len(ordered_points) != 4:
                ordered_points = cv2.boxPoints(cv2.minAreaRect(ordered_points)) 
            
            # 强制顺时针 
            edge_sum = sum(
                (ordered_points[i+1][0] - ordered_points[i][0]) * 
                (ordered_points[i+1][1] + ordered_points[i][1]) 
                for i in range(3)
            )
            if edge_sum <= 0:
                ordered_points = ordered_points[::-1]
            
            # 归一化 
            norm_points = ordered_points / np.array([img_w, img_h])
            txt_lines.append( 
                f"{class_list.index(label)} " + 
                " ".join(f"{p:.8f}" for p in norm_points.flatten()) 
            )
        
        # 多边形处理（新增部分）
        elif shape["shape_type"] == "polygon":
            # 确保多边形点集有效
            if len(points) < 3:
                continue
            
            # 计算最小外接旋转矩形
            rect = cv2.minAreaRect(points.astype(np.float32))
            box_points = cv2.boxPoints(rect)
            
            # 确保坐标不超出图像范围
            box_points[:, 0] = np.clip(box_points[:, 0], 0, img_w - 1)
            box_points[:, 1] = np.clip(box_points[:, 1], 0, img_h - 1)
            
            # 顶点顺序标准化（顺时针）
            box_points = ensure_clockwise(box_points)
            
            # 归一化处理
            norm_points = box_points / np.array([img_w, img_h])
            txt_lines.append(
                f"{class_list.index(label)} " +
                " ".join(f"{p:.8f}" for p in norm_points.flatten())
            )
    
    # 写入文件 
    os.makedirs(output_dir, exist_ok=True)
    txt_path = os.path.join(output_dir, Path(image_path).stem + ".txt")
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(txt_lines))

In [6]:
import glob 
from tqdm import tqdm  # 进度条支持 
 
def batch_convert(image_dir, json_dir, output_dir, class_list):
    """批量处理目录下所有匹配的图片和JSON文件"""
    # 获取所有图片路径（支持多种格式）
    image_paths = glob.glob(os.path.join(image_dir,  "*.jpg")) + \
                 glob.glob(os.path.join(image_dir,  "*.png"))
    
    # 进度条可视化 
    for img_path in tqdm(image_paths, desc="转换进度"):
        # 构建对应JSON路径 
        json_path = os.path.join(json_dir,  os.path.basename(img_path).split('.')[0]  + ".json")
        
        if not os.path.exists(json_path): 
            print(f"警告：缺失标注文件 {json_path}")
            continue 
            
        # 调用单文件处理函数 
        convert_to_yolov8(
            image_path=img_path,
            json_path=json_path,
            output_dir=output_dir,
            class_list=class_list 
        )

In [7]:
if __name__ == "__main__":
    # 配置路径 
    image_dir = "D:/project/project_drive/zhang_drive/train_yolov8_obb/newdata/image"
    json_dir = "D:/project/project_drive/zhang_drive/train_yolov8_obb/newdata/mydata"
    output_dir = "D:/project/project_drive/zhang_drive/train_yolov8_obb/newdata/labels"
    class_list = ["face", "belt","phone", "smoke", "food","items"]  # 或从文件读取 
    # 执行批量转换 
    batch_convert(image_dir, json_dir, output_dir, class_list)
    
    print(f"批量转换完成！共处理 {len(os.listdir(image_dir))}  张图片")

转换进度: 100%|██████████| 138/138 [00:00<00:00, 407.50it/s]

批量转换完成！共处理 138  张图片


### 验证标签（有问题不能停止）

In [ ]:
import cv2
import numpy as np

# 示例：绘制 OBB 框
img = cv2.imread("/home/zhuofei/linux_data/liujunyan/DMS/zhang_drive/badbehavior_classes6/image/IRCamera_ID00011_20222017_女_21_DZ01_hat_000000011.png")
h, w = img.shape[:2]
obb_line = "1 0.87445107 0.70505134 0.91142902 0.70264630 0.90824966 0.54814860 0.87127171 0.55055364"  # 示例 OBB 标签
parts = list(map(float, obb_line.split()))
points = np.array([(x * w, y * h) for x, y in zip(parts[1::2], parts[2::2])], dtype=np.int32)
cv2.polylines(img, [points], isClosed=True, color=(0, 255, 0), thickness=2)
cv2.imshow("OBB",  img)
key = cv2.waitKey(5000)   # 5秒超时自动关闭 
if key == -1:  
    cv2.destroyAllWindows() 

### PNG转JPG

In [ ]:
import cv2
import os
from pathlib import Path

# 输入和输出文件夹
input_dir = Path("文件路径")  # 替换为你的PNG文件夹路径，文件夹里图片命名不要有中文
output_dir = Path("文件路径")  # JPG输出文件夹
output_dir.mkdir(parents=True, exist_ok=True)  # 创建输出文件夹

# 遍历所有PNG文件
for png_file in input_dir.glob("*.png"):
    # 读取PNG图片
    img = cv2.imread(str(png_file))
    if img is None:
        continue  # 跳过无法读取的文件

    # 构造输出路径（将后缀改为.jpg）
    jpg_file = output_dir / f"{png_file.stem}.jpg"
    
    # 保存为JPG（质量参数可选，范围0-100，默认95）
    cv2.imwrite(str(jpg_file), img, [int(cv2.IMWRITE_JPEG_QUALITY), 90])

print(f"转换完成！JPG文件保存在：{output_dir}")

转换完成！JPG文件保存在：/home/zhuofei/linux_data/liujunyan/DMS/zhang_drive/bad_class6_new/images_jpg
